<a href="https://colab.research.google.com/github/sergekamanzi/Chat-Bot-/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install gradio

  Using cached gradio-5.31.0-py3-none-any.whl.metadata (16 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00


In [11]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import gradio as gr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

# Define the question-answer data
data = {
    "What was the significance of 1957 in Rwanda?": "In 1957, the Hutu Manifesto was published by Hutu intellectuals demanding political reform and majority rule, marking a turning point in ethnic tensions.",
    "What happened during the Rwandan Revolution of 1959?": "The Rwandan Revolution in 1959 led to the end of the Tutsi monarchy and the rise of a Hutu-led republic. It triggered ethnic violence and the exile of many Tutsis.",
    "When did Rwanda gain independence?": "Rwanda gained independence from Belgium on July 1, 1962, with Grégoire Kayibanda becoming the first president.",
    "Who was Grégoire Kayibanda?": "Grégoire Kayibanda was Rwanda's first president from 1962 to 1973. He led the country after independence and promoted Hutu nationalism.",
    "What happened in 1973 in Rwanda?": "In 1973, Major General Juvénal Habyarimana overthrew Grégoire Kayibanda in a military coup and established a one-party state under the MRND party.",
    "Who was Juvénal Habyarimana?": "Juvénal Habyarimana was Rwanda's second president, ruling from 1973 until his assassination in 1994. His regime was marked by ethnic favoritism and authoritarianism.",
    "What caused the 1994 Genocide?": "The 1994 Genocide against the Tutsi was caused by long-standing ethnic tensions, political instability, and extremist propaganda. The assassination of President Habyarimana triggered the mass killings.",
    "What happened during the 1994 Genocide in Rwanda?": "From April to July 1994, approximately 800,000 Tutsi and moderate Hutu were killed in a genocide perpetrated by Hutu extremists.",
    "Who ended the 1994 Genocide in Rwanda?": "The Rwandan Patriotic Front (RPF), led by Paul Kagame, ended the genocide in July 1994 after capturing Kigali and taking control of the country.",
    "What is the Rwandan Patriotic Front (RPF)?": "The RPF is a political and military organization that ended the 1994 Genocide and has led the Rwandan government since then. It was founded by Tutsi exiles in Uganda.",
    "Who is Paul Kagame?": "Paul Kagame is Rwanda's current president and former military leader of the RPF. He played a key role in ending the 1994 Genocide and has been president since 2000.",
    "What happened in Rwanda after the Genocide?": "After the Genocide, Rwanda focused on national unity, justice through Gacaca courts, reconstruction, and economic development under RPF leadership.",
    "What are Gacaca courts in Rwanda?": "Gacaca courts were community-based courts used to try suspects of the 1994 Genocide, aimed at truth-telling, justice, and reconciliation.",
    "When did Paul Kagame become president?": "Paul Kagame officially became president of Rwanda in 2000 after Pasteur Bizimungu resigned.",
    "What has Rwanda achieved since 2000?": "Since 2000, Rwanda has made progress in healthcare, education, digital transformation, gender equality, and economic development.",
    "What is Vision 2020 in Rwanda?": "Vision 2020 was a national development plan launched by the Rwandan government to transform the country into a middle-income economy by the year 2020.",
    "What replaced Vision 2020 in Rwanda?": "Vision 2050 replaced Vision 2020 and aims to make Rwanda an upper-middle-income country by 2035 and a high-income country by 2050.",
    "What is Umuganda in Rwanda?": "Umuganda is a monthly national community service day in Rwanda where citizens clean streets, plant trees, and build public infrastructure.",
    "What is Rwanda known for today?": "Rwanda is known for its clean cities, rapid development, eco-tourism, commitment to reconciliation, and being one of the safest countries in Africa.",
    "What is Kwibuka in Rwanda?": "Kwibuka is an annual commemoration of the 1994 Genocide against the Tutsi. It means 'to remember' in Kinyarwanda and begins every April 7."
}

# Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Extract questions and answers
questions = list(data.keys())
answers = list(data.values())

# Encode the questions
question_embeddings = model.encode(questions, convert_to_tensor=False)
question_embeddings = np.array(question_embeddings).astype('float32')

# Create a FAISS index
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(question_embeddings)

# Lists for detecting greetings, gratitude, farewells, emotions, and follow-ups
GREETINGS = ["hello", "hi", "hey", "greetings", "good morning", "good afternoon", "good evening"]
GRATITUDE = ["thank you", "thanks", "appreciate it", "thank you so much", "thanks a lot"]
FAREWELLS = ["bye", "goodbye", "see you", "farewell", "take care"]
FOLLOW_UPS = ["tell me more", "more", "go on", "continue"]

# Emotion keyword lists
HAPPINESS = ["happy", "glad", "excited", "joyful", "pleased"]
SADNESS = ["sad", "heartbreaking", "tragic", "depressing", "devastating"]
ANGER = ["angry", "frustrated", "outraged", "mad", "furious"]
FEAR = ["scared", "terrified", "afraid", "horrified", "fearful"]
SURPRISE = ["shocked", "amazed", "surprised", "astonished", "stunned"]

def detect_emotion(query):
    query_lower = query.lower().strip()
    if any(word in query_lower for word in HAPPINESS):
        return "happiness", "I’m glad to hear that brings you happiness."
    elif any(word in query_lower for word in SADNESS):
        return "sadness", "I understand, that can be truly heartbreaking."
    elif any(word in query_lower for word in ANGER):
        return "anger", "I can see why that might make you feel angry."
    elif any(word in query_lower for word in FEAR):
        return "fear", "I can understand why that might feel terrifying."
    elif any(word in query_lower for word in SURPRISE):
        return "surprise", "That’s quite surprising, isn’t it?"
    return None, ""

def is_greeting(query):
    query_lower = query.lower().strip()
    return any(greeting in query_lower for greeting in GREETINGS)

def is_gratitude(query):
    query_lower = query.lower().strip()
    return any(gratitude in query_lower for gratitude in GRATITUDE)

def is_farewell(query):
    query_lower = query.lower().strip()
    return any(farewell in query_lower for farewell in FAREWELLS)

def is_follow_up(query):
    query_lower = query.lower().strip()
    return any(follow_up in query_lower for follow_up in FOLLOW_UPS)

# Prepare data for LSTM training
all_text = questions + answers + [f"User: {q} Bot: {a}" for q, a in zip(questions, answers)]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_text)
vocab_size = len(tokenizer.word_index) + 1

# Create sequences for training (simplified example)
sequences = tokenizer.texts_to_sequences(all_text)
max_length = max(len(seq) for seq in sequences)

# Pad sequences for X
X = pad_sequences(sequences[:-1], maxlen=max_length, padding='post')

# Prepare y by taking the next word index for each sequence in X
# This involves taking the token *after* the sequence in X.
# We need to make sure the target sequences are not empty after slicing.
y_sequences = [seq[max_length] if len(seq) > max_length else 0 for seq in sequences[1:]] # Get the next word index, handle shorter sequences

# Use to_categorical on the next word indices
y = tf.keras.utils.to_categorical(y_sequences, num_classes=vocab_size)


# Build and train LSTM model
lstm_model = Sequential([
    Embedding(vocab_size, 64, input_length=max_length),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dense(vocab_size, activation='softmax')
])
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam')
# Ensure the shape of X matches the input_length
# We need X to have shape (num_samples, max_length) and y to have shape (num_samples, vocab_size)
# Check shapes before training
# print(f"Shape of X: {X.shape}")
# print(f"Shape of y: {y.shape}")
lstm_model.fit(X, y, epochs=50, batch_size=1, verbose=0)  # Reduced verbosity for Colab

def generate_lstm_response(history_text):
    # Tokenize the history text
    sequence = tokenizer.texts_to_sequences([history_text])
    # Check if the sequence is empty or has length 0
    if not sequence or not sequence[0]:
        return "I can provide more details. What topic would you like to explore?"

    # Pad the sequence
    padded = pad_sequences(sequence, maxlen=max_length, padding='post')
    # Predict the next token
    prediction = lstm_model.predict(padded, verbose=0)
    predicted_index = np.argmax(prediction)
    # Convert index back to word
    word = tokenizer.index_word.get(predicted_index, "details") # Use .get with a default value
    return word


def chatbot(query, history):
    # Initialize history if None
    if history is None:
        history = []

    # Detect emotion
    emotion, emotion_response = detect_emotion(query)

    # Check for greetings
    if is_greeting(query):
        response = "Hello! I'm here to assist you with questions about Rwandan history. What would you like to learn about today?"
        history.append(("User: " + query, "Bot: " + response))
        return response, history

    # Check for gratitude
    if is_gratitude(query):
        response = "You're welcome! I'm glad I could help. Would you like to know more about Rwandan history?"
        history.append(("User: " + query, "Bot: " + response))
        return response, history

    # Check for farewells
    if is_farewell(query):
        response = "Goodbye! Feel free to come back if you have more questions about Rwandan history."
        history.append(("User: " + query, "Bot: " + response))
        return response, history

    # Check for follow-up questions
    if is_follow_up(query) and history:
        # Construct history text from the last turn
        # This might be too simple for complex follow-ups, could consider last N turns
        last_turn = history[-1]
        history_text = last_turn[0] + " " + last_turn[1] # Combine user query and bot response from the last turn
        lstm_response_word = generate_lstm_response(history_text)
        # Generate a more coherent response using the predicted word
        response = f"Based on our conversation, perhaps you're interested in '{lstm_response_word}'. Would you like to know more about Rwandan history?"
        history.append(("User: " + query, "Bot: " + response))
        return response, history

    # Encode the input query
    query_embedding = model.encode([query], convert_to_tensor=False).astype('float32')

    # Search for the closest question
    k = 1  # Number of nearest neighbors
    distances, indices = index.search(query_embedding, k)

    # Get the distance to the closest match
    distance = distances[0][0]

    # Set a threshold for out-of-domain detection
    DISTANCE_THRESHOLD = 1.5 # This threshold might need tuning

    if distance > DISTANCE_THRESHOLD:
        response = "I'm sorry, I don't have information on that topic. I can help with questions about Rwandan history. What would you like to know?"
        history.append(("User: " + query, "Bot: " + response))
        return response, history

    # Get the most similar question and its answer
    matched_question = questions[indices[0][0]]
    matched_answer = answers[indices[0][0]]

    # Add empathetic response if emotion is detected
    if emotion:
        response = f"{emotion_response} Here’s what I found: {matched_answer} Would you like to know more about Rwandan history?"
    else:
        response = f"Here’s what I found: {matched_answer} Would you like to know more about Rwandan history?"

    history.append(("User: " + query, "Bot: " + response))
    return response, history

# Create a Gradio interface with state to maintain history
inputs = [
    gr.Textbox(lines=2, placeholder="Ask a question about Rwandan history..."),
    gr.State(value=[])
]
outputs = [
    gr.Textbox(label="Response"),
    gr.State()
]

iface = gr.Interface(
    fn=chatbot,
    inputs=inputs,
    outputs=outputs,
    title="Rwandan History Chatbot",
    description="Ask questions about Rwandan history, and the chatbot will respond based on the provided knowledge base."
)

# Test the chatbot with sample queries
test_queries = [
    "Hello",  # Greeting
    "The 1994 genocide is so heartbreaking",  # Sadness
    "Tell me more",  # Follow-up
    "I’m so happy to learn about Rwanda’s progress",  # Happiness
    "What happened in 1973 in Rwanda?",  # In-domain
    "That’s shocking!",  # Surprise
    "Thanks a lot",  # Gratitude
    "Goodbye",  # Farewell
]

print("Testing the chatbot with sample queries:")
history = []
for query in test_queries:
    response, history = chatbot(query, history)
    print(f"Query: {query}")
    print(f"Response: {response}\n")

# Launch the Gradio interface
iface.launch()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Testing the chatbot with sample queries:
Query: Hello
Response: Hello! I'm here to assist you with questions about Rwandan history. What would you like to learn about today?

Query: The 1994 genocide is so heartbreaking
Response: I understand, that can be truly heartbreaking. Here’s what I found: The 1994 Genocide against the Tutsi was caused by long-standing ethnic tensions, political instability, and extremist propaganda. The assassination of President Habyarimana triggered the mass killings. Would you like to know more about Rwandan history?

Query: Tell me more
Response: Based on our conversation, perhaps you're interested in 'details'. Would you like to know more about Rwandan history?

Query: I’m so happy to learn about Rwanda’s progress
Response: I’m glad to hear that brings you happiness. Here’s what I found: Since 2000, Rwanda has made progress in healthcare, education, digital transformation, gender equality, and economic development. Would you like to know more about Rwandan

KeyboardInterrupt: 